In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error


In [2]:

csv_file_path = 'train.csv' 
train_df = pd.read_csv(csv_file_path)
print(train_df.head())
print(f"\n总样本数: {len(train_df)}")


image_dir = './ot'
if not os.path.isdir(image_dir):
    print(f"警告：图像目录 '{image_dir}' 不存在。请确保路径正确，并且包含.npy文件。")

# 3. 尝试加载第一个 .npy 文件来查看其结构
if len(train_df) > 0 and os.path.isdir(image_dir):
    first_image_name = train_df.loc[0, 'id'] # 获取第一个样本的 .npy 文件名
    first_image_path = os.path.join(image_dir, first_image_name)

    try:
        print(f"\n尝试加载图像文件: {first_image_path}")
        image_data = np.load(first_image_path)
        print("图像文件加载成功！")
        print(f"图像数据类型: {image_data.dtype}")
        print(f"图像数据形状 (shape): {image_data.shape}")
        # 根据我们之前的讨论，我们期望形状类似于 (高度, 宽度, 100)
        if len(image_data.shape) == 3 and image_data.shape[2] == 100:
            print("图像的波段数量为100，符合预期。")
        else:
            print(f"警告：图像形状 {image_data.shape} 与预期的 (高度, 宽度, 100) 不完全一致，请检查。")

    except FileNotFoundError:
        print(f"错误：未找到图像文件 {first_image_path}。请确保 'image_dir' 设置正确且文件存在。")
    except Exception as e:
        print(f"加载图像文件 {first_image_path} 时发生错误: {e}")
else:
    if len(train_df) == 0:
        print("train_df 为空，无法加载图像。")
    # 如果目录不存在，上面已经给过警告了

               id  label
0   sample697.npy      7
1    sample54.npy     81
2  sample2270.npy      4
3  sample1401.npy     99
4  sample1901.npy     43

总样本数: 2177

尝试加载图像文件: ./ot\sample697.npy
图像文件加载成功！
图像数据类型: uint16
图像数据形状 (shape): (128, 128, 125)
警告：图像形状 (128, 128, 125) 与预期的 (高度, 宽度, 100) 不完全一致，请检查。


一点发现：跟数据集描述的不同，训练集的2217条数据和545条数据中都有不符合预期的（125，57，125）大小的数据，我们需要进行填充。考虑到我们的目标是预测病害百分比，以及我们选用的特征（各波段平均反射率）具体的措施： 在您的 `extract_features` 函数中，当遇到宽度为57的图像时，可以对每个波段（经过125选100波段之后）应用 反射填充或者是均值填充来进行两侧填充，将其宽度扩展到128。然后再计算这100个（已填充到128宽度）波段各自的平均值。

In [3]:

TARGET_HEIGHT = 128
TARGET_WIDTH = 128
ORIGINAL_BANDS = 125 # 原始文件中的波段数

# 3. 定义波段选择的切片 (选择索引10到109的波段，共100个)
band_selection_slice = slice(10, 110)
NUM_SELECTED_BANDS = 100 # band_selection_slice.stop - band_selection_slice.start

def process_and_extract_features(image_path, target_height, target_width, band_slice):
    """
    加载高光谱图像，对需要填充的图像进行填充，选择波段，并计算平均反射率。
    """
    try:
        image_data = np.load(image_path) # 期望形状 (h, w, 125)
        
        current_height, current_width, current_bands = image_data.shape

        # 检查基本维度是否符合预期（高度和原始波段数）
        if current_height != target_height or current_bands != ORIGINAL_BANDS:
            print(f"警告：文件 {os.path.basename(image_path)} 的形状 ({current_height}, {current_width}, {current_bands}) "
                  f"与预期的高度 {target_height} 或原始波段数 {ORIGINAL_BANDS} 不符。将跳过此文件。")
            return None

        # 如果宽度不是目标宽度，则进行填充 (例如，原始宽度为57，目标为128)
        if current_width != target_width:
            if current_width < target_width: # 只处理宽度不足的情况
                pad_total = target_width - current_width
                pad_left = pad_total // 2
                pad_right = pad_total - pad_left
                
                # 对宽度维度进行填充，其他维度不填充
                # ((上下填充), (左右填充), (通道前后填充))
                padding_config = ((0, 0), (pad_left, pad_right), (0, 0))
                
                # 使用 'reflect' 模式填充
                # 您可以将 'reflect' 更改为 'symmetric' 来测试不同效果
                padded_image_data = np.pad(image_data, padding_config, mode='reflect')
                # print(f"填充文件 {os.path.basename(image_path)} from {image_data.shape} to {padded_image_data.shape}")
            else: # 如果图像宽度大于目标宽度，可以选择裁剪或跳过
                print(f"警告：文件 {os.path.basename(image_path)} 的宽度 {current_width} 大于目标宽度 {target_width}。将跳过此文件。")
                return None
        else:
            padded_image_data = image_data # 不需要填充

        # 波段选择 (此时 padded_image_data 的形状应为 [target_height, target_width, ORIGINAL_BANDS])
        selected_bands_data = padded_image_data[:, :, band_slice] # 形状变为 [h, w, NUM_SELECTED_BANDS]
        
        # 计算每个选定波段的平均反射率
        mean_reflectance = np.mean(selected_bands_data, axis=(0, 1)) # 形状变为 [NUM_SELECTED_BANDS,]
        
        return mean_reflectance
        
    except FileNotFoundError:
        print(f"错误：处理时未找到文件 {image_path}")
        return None
    except Exception as e:
        print(f"错误：处理文件 {image_path} 时发生: {e}")
        return None
    
def process_and_extract_features_mean_pad(image_path, target_height, target_width, original_bands_count, band_slice):
    """
    加载高光谱图像，对需要填充的图像使用其各波段自身均值进行填充，
    然后选择波段，并计算平均反射率。
    """
    try:
        image_data = np.load(image_path) # 期望形状 (h, w, original_bands_count)
        
        current_height, current_width, current_bands = image_data.shape

        if current_height != target_height or current_bands != original_bands_count:
            print(f"警告：文件 {os.path.basename(image_path)} 的形状 ({current_height}, {current_width}, {current_bands}) "
                  f"与预期的高度 {target_height} 或原始波段数 {original_bands_count} 不符。将跳过此文件。")
            return None

        processed_image_data = np.zeros((target_height, target_width, original_bands_count), dtype=image_data.dtype)

        if current_width != target_width:
            if current_width < target_width: # 只处理宽度不足的情况
                pad_total = target_width - current_width
                pad_left = pad_total // 2
                pad_right = pad_total - pad_left
                
                padding_config_2d = ((0, 0), (pad_left, pad_right)) # 只针对H, W维度中的W

                # 逐个波段进行均值填充
                for i in range(original_bands_count):
                    band_original_data = image_data[:, :, i] # 单个波段 (H, current_W)
                    band_mean = np.mean(band_original_data)   # 计算该波段原始有效区域的均值
                    
                    # 使用计算得到的均值进行常数填充
                    padded_band = np.pad(band_original_data, padding_config_2d, mode='constant', constant_values=band_mean)
                    processed_image_data[:, :, i] = padded_band
                
                # print(f"均值填充文件 {os.path.basename(image_path)} from {image_data.shape} to {processed_image_data.shape}")

            else: # 如果图像宽度大于目标宽度
                print(f"警告：文件 {os.path.basename(image_path)} 的宽度 {current_width} 大于目标宽度 {target_width}。将跳过此文件。")
                return None
        else: # 宽度已经是目标宽度，无需填充
            processed_image_data = image_data
        
        # 波段选择 (此时 processed_image_data 的形状应为 [target_height, target_width, original_bands_count])
        selected_bands_data = processed_image_data[:, :, band_slice] # 形状变为 [h, w, NUM_SELECTED_BANDS]
        
        # 计算每个选定波段的平均反射率
        mean_reflectance = np.mean(selected_bands_data, axis=(0, 1)) # 形状变为 [NUM_SELECTED_BANDS,]
        
        return mean_reflectance
        
    except FileNotFoundError:
        print(f"错误：处理时未找到文件 {image_path}")
        return None
    except Exception as e:
        print(f"错误：处理文件 {image_path} 时发生: {e}")
        return None
    

# 准备存储特征和标签的列表
X_list = []
y_list = []
skipped_files_count = 0

print(f"\n开始为所有 {len(train_df)} 个训练样本提取特征...")
for index, row in tqdm(train_df.iterrows(), total=len(train_df)):
    image_name = row['id']
    label = row['label']
    current_image_path = os.path.join(image_dir, image_name)
    
    features = process_and_extract_features(current_image_path, 
                                            TARGET_HEIGHT, 
                                            TARGET_WIDTH, 
                                            band_selection_slice)
    
    if features is not None and features.shape == (NUM_SELECTED_BANDS,):
        X_list.append(features)
        y_list.append(label)
    else:
        # print(f"警告：未能为样本 {image_name} 提取有效特征，已跳过。")
        skipped_files_count += 1


# 将列表转换为 NumPy 数组
X_train = np.array(X_list)
y_train = np.array(y_list)

print(f"\n特征提取完成！")
if skipped_files_count > 0:
    print(f"注意：共有 {skipped_files_count} 个文件因形状问题或处理错误被跳过。")
print(f"X_train 的形状: {X_train.shape}") # 应该为 (成功处理的样本数, 100)
print(f"y_train 的形状: {y_train.shape}") # 应该为 (成功处理的样本数,)


开始为所有 2177 个训练样本提取特征...


 43%|████▎     | 934/2177 [00:06<00:08, 147.28it/s]

错误：处理文件 ./ot\sample2451.npy 时发生: cannot reshape array of size 1785856 into shape (128,128,125)


100%|██████████| 2177/2177 [00:15<00:00, 142.00it/s]


特征提取完成！
注意：共有 1 个文件因形状问题或处理错误被跳过。
X_train 的形状: (2176, 100)
y_train 的形状: (2176,)


In [4]:
# 检查 X_train 和 y_train 是否有数据
if X_train.shape[0] > 0 and y_train.shape[0] > 0 and X_train.shape[0] == y_train.shape[0]:
    # 进行数据划分
    # test_size=0.2 表示验证集占20%
    # random_state 是一个种子，确保每次划分结果一致，方便复现
    X_train_cv, X_val_cv, y_train_cv, y_val_cv = train_test_split(
        X_train,
        y_train,
        test_size=0.2,
        random_state=42 # 您可以选择任何整数作为随机种子
    )

    print("\n数据划分完成！")
    print(f"新的训练集 X_train_cv 的形状: {X_train_cv.shape}")
    print(f"新的训练集标签 y_train_cv 的形状: {y_train_cv.shape}")
    print(f"验证集 X_val_cv 的形状: {X_val_cv.shape}")
    print(f"验证集标签 y_val_cv 的形状: {y_val_cv.shape}")

    # 2176 * 0.8 = 1740.8 (约 1740 或 1741)
    # 2176 * 0.2 = 435.2 (约 435 或 436)
    # scikit-learn 会自动处理取整

else:
    print("错误：X_train 或 y_train 为空或样本数量不匹配，无法进行数据划分。")
    print("请确保之前的特征提取步骤已成功生成 X_train 和 y_train。")


数据划分完成！
新的训练集 X_train_cv 的形状: (1740, 100)
新的训练集标签 y_train_cv 的形状: (1740,)
验证集 X_val_cv 的形状: (436, 100)
验证集标签 y_val_cv 的形状: (436,)


In [5]:

print(f"您当前使用的 XGBoost 版本是: {xgb.__version__}")

# 创建一个临时的 XGBRegressor 实例，以便查看其 fit 方法的帮助文档
temp_model = xgb.XGBRegressor()

# 打印 fit 方法的帮助信息，这将显示它接受哪些参数
help(temp_model.fit)

您当前使用的 XGBoost 版本是: 2.0.3
Help on method fit in module xgboost.sklearn:

fit(X: Any, y: Any, *, sample_weight: Optional[Any] = None, base_margin: Optional[Any] = None, eval_set: Optional[Sequence[Tuple[Any, Any]]] = None, eval_metric: Union[str, Sequence[str], Callable[[numpy.ndarray, xgboost.core.DMatrix], Tuple[str, float]], NoneType] = None, early_stopping_rounds: Optional[int] = None, verbose: Union[bool, int, NoneType] = True, xgb_model: Union[xgboost.core.Booster, ForwardRef('XGBModel'), str, NoneType] = None, sample_weight_eval_set: Optional[Sequence[Any]] = None, base_margin_eval_set: Optional[Sequence[Any]] = None, feature_weights: Optional[Any] = None, callbacks: Optional[Sequence[xgboost.callback.TrainingCallback]] = None) -> 'XGBModel' method of xgboost.sklearn.XGBRegressor instance
    Fit gradient boosting model.

    Note that calling ``fit()`` multiple times will cause the model object to be
    re-fit from scratch. To resume training from a previous checkpoint, explici

In [7]:
if X_train_cv.shape[0] > 0 and X_val_cv.shape[0] > 0:
    print("\n开始训练XGBoost回归模型...")

    # 1. 定义早停回调
    early_stopping_callback = xgb.callback.EarlyStopping(
        rounds=50,
        metric_name='rmse',
        data_name='validation_1', # 对应 eval_set 中的第二个元素
        save_best=True # 推荐开启，这样模型会自动是最佳状态
    )

    # 2. 初始化 XGBoost 回归器，并将 callbacks 传入构造函数
    xgb_regressor_fixed = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='rmse',
        tree_method='hist',
        callbacks=[early_stopping_callback] # <--- callbacks 移到此处
    )

    # 3. 训练模型
    # 注意：fit 方法中不再需要 callbacks 或 early_stopping_rounds 参数
    xgb_regressor_fixed.fit(
        X_train_cv,
        y_train_cv,
        eval_set=[(X_train_cv, y_train_cv), (X_val_cv, y_val_cv)],
        verbose=100
    )

    print("\n模型训练完成！")

    # 4. 在验证集上进行预测并评估
    # 由于 save_best=True (在 EarlyStopping 回调中)，模型已经是最佳迭代次数的状态
    y_pred_val = xgb_regressor_fixed.predict(X_val_cv)
    
    rmse_val = np.sqrt(mean_squared_error(y_val_cv, y_pred_val))
    print(f"验证集上的均方根误差 (RMSE): {rmse_val:.4f}")
    
    # 获取最佳迭代次数
    # 当使用 EarlyStopping 回调且 save_best=True 时，
    # best_iteration 通常可以直接从模型对象获取，或者回调对象自身也会存储
    if hasattr(xgb_regressor_fixed, 'best_iteration'):
         print(f"模型在第 {xgb_regressor_fixed.best_iteration +1 } 轮达到最佳。")
    elif hasattr(early_stopping_callback, 'best_iteration'): # 回调对象也可能存储
         print(f"模型在第 {early_stopping_callback.best_iteration + 1} 轮达到最佳。")
    else:
        # 如果没有直接的 best_iteration 属性，
        # 可以通过 xgb_regressor_fixed.get_booster().best_iteration 获取
        # 但这取决于具体的XGBoost版本和回调实现细节
        try:
            print(f"模型在第 {xgb_regressor_fixed.get_booster().best_iteration + 1} 轮达到最佳。")
        except Exception:
            print("模型已根据早停规则训练至最佳状态 (具体轮数需从日志或回调中确认)。")


else:
    print("错误：训练集 (X_train_cv) 或验证集 (X_val_cv) 为空，无法训练模型。")


开始训练XGBoost回归模型...
[0]	validation_0-rmse:29.40647	validation_1-rmse:29.18739


[51]	validation_0-rmse:26.16207	validation_1-rmse:29.62095

模型训练完成！
验证集上的均方根误差 (RMSE): 29.1847
模型在第 2 轮达到最佳。
